In [1]:
from __future__ import unicode_literals

import re

import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from tensorflow.keras.layers import GRU, Bidirectional, Dense, Dropout, Embedding,LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-11-28 17:38:08.204824: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-28 17:38:08.247089: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-28 17:38:08.247118: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-28 17:38:08.248273: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugi

In [2]:
# Download NLTK resources
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
df = pd.read_csv("dataset/semeval-2017-train.csv", on_bad_lines="skip", sep="\t")
df_t = pd.read_csv("dataset/semeval-2017-test.csv", on_bad_lines="skip", sep="\t")

In [4]:
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove punctuation and special characters
    text = re.sub(r"[^\w\s]", "", text)

    # Tokenization (you can customize this based on your needs)
    tokens = text.split()

    # Remove stopwords
    stop_words = set(stopwords.words("english"))
    tokens = [word for word in tokens if word not in stop_words]

    # Join tokens back into a string
    preprocessed_text = " ".join(tokens)

    return preprocessed_text

In [5]:
# Apply preprocessing to the comments
df["processed_text"] = df["text"].apply(preprocess_text)
df_t["processed_text"] = df_t["text"].apply(preprocess_text)
df['label'] = df['label'].replace({0:1})
df_t['label'] = df_t['label'].replace({0:1})

In [6]:
# Tokenization
texts = df["processed_text"]
labels = np.array(df["label"])

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

vocab_size = len(tokenizer.word_index) + 1
max_length = max(len(seq) for seq in sequences)
embedding_dim = 100

padded_sequences = pad_sequences(sequences, maxlen=max_length)

In [18]:
# Model definition
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
# model.add(GRU(128))
model.add(LSTM(128))
model.add(Dropout(0.8))
# model.add(Dense(units=2, activation='softmax'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = []
# Training
step = 12000
for i in range(0, 48000, step):
    h = model.fit(padded_sequences[i:i + step], labels[i:i + step], batch_size=len(padded_sequences[i:i + step]), epochs=10)
    history += h.history['accuracy']


Epoch 1/10
1/1 [==============================] - 10s 10s/step - loss: 0.6933 - accuracy: 0.4222
Epoch 2/10
1/1 [==============================] - 9s 9s/step - loss: 0.6881 - accuracy: 0.7484
Epoch 3/10
1/1 [==============================] - 9s 9s/step - loss: 0.6821 - accuracy: 0.8432
Epoch 4/10
1/1 [==============================] - 9s 9s/step - loss: 0.6738 - accuracy: 0.8549
Epoch 5/10
1/1 [==============================] - 8s 8s/step - loss: 0.6632 - accuracy: 0.8558
Epoch 6/10
1/1 [==============================] - 9s 9s/step - loss: 0.6479 - accuracy: 0.8558
Epoch 7/10
1/1 [==============================] - 8s 8s/step - loss: 0.6235 - accuracy: 0.8558
Epoch 8/10
1/1 [==============================] - 9s 9s/step - loss: 0.5996 - accuracy: 0.8558
Epoch 9/10
1/1 [==============================] - 9s 9s/step - loss: 0.6591 - accuracy: 0.8558
Epoch 10/10
1/1 [==============================] - 9s 9s/step - loss: 0.6178 - accuracy: 0.8558
Epoch 1/10
1/1 [==============================]

In [19]:
# model.fit(padded_sequences[45000:49000], labels[45000:49000], batch_size=len(padded_sequences[45000:49000]), epochs=10)
# Testing
test_texts = df_t['processed_text']
true_labels = np.array(df_t['label'])

test_sequences = tokenizer.texts_to_sequences(test_texts)
padded_test_sequences = pad_sequences(test_sequences, maxlen=max_length)
predictions = model.predict(padded_test_sequences)

print(predictions)

375/375 [==============================] - 16s 41ms/step
[[0.45613053]
 [0.40381715]
 [0.4670435 ]
 ...
 [0.8282494 ]
 [0.69376713]
 [0.83340305]]


In [21]:
binary_predictions = [1 if (p[0]) > 0.5 else -1 for p in predictions]
acc = accuracy_score(true_labels, binary_predictions)
print(acc)
model.save('7207.h5')

0.7207147031810971


/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
